In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import fastai

In [ ]:
from fastai.vision.all import *

In [ ]:
fastai.__version__

In [ ]:


!pip install -q git+https://github.com/tmabraham/UPIT.git



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import glob
import tqdm
import os
import PIL
from fastai.vision.all import *
from fastai.basics import *
from upit.models.cyclegan import *
from upit.inference.cyclegan import *
from upit.train.cyclegan import *
from upit.data.unpaired import *

In [ ]:
import os
import sys
import gc
from PIL import Image as immg
import shutil

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:


path = Path('../input/datasetfast/halfdata/images');
for p in path.ls():
    print(p)



In [ ]:
ct=path/'trainA'
mri=path/'trainB'

In [ ]:


ct_files = get_image_files(ct)
mri_files = get_image_files(mri)

In [ ]:
fig = plt.figure(1,figsize=(20,15))
columns = 4
rows = 3
for i in range(rows*columns):
    num = random.randint(2,300)
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(immg.open(mri_files[num]))
    plt.axis('off')
plt.subplots_adjust(wspace=0, hspace=0.1)
plt.show()

In [ ]:
fig = plt.figure(1,figsize=(20,15))
columns = 4
rows = 3
for i in range(rows*columns):
    num = random.randint(2,300)
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(immg.open(ct_files[num]),cmap='gray')
    plt.axis('off')
plt.subplots_adjust(wspace=0, hspace=0.1)
plt.show()

In [ ]:
dls = get_dls( ct , mri ,load_size=256,crop_size=256)


In [ ]:


dls.show_batch(figsize=(20,8))



In [ ]:


cycle_gan = CycleGAN(3,3,64, gen_blocks=9)
learn = cycle_learner(dls, cycle_gan,opt_func=partial(Adam,mom=0.5,sqr_mom=0.999))



In [ ]:


learn.lr_find()



In [ ]:


gc.collect()



In [ ]:


learn.fit_flat_lin(15 ,15 ,1e-4)



In [ ]:
learn.save('ct2mri');

In [ ]:
learn.recorder.plot_loss(with_valid=False)

In [ ]:


b = dls.train.one_batch()
_,_,preds = learn.get_preds(dl=[b], with_decoded=True)



In [ ]:


dls.show_batch((b[0], b[1]), max_n=8, show=True,figsize=(20,8))



In [ ]:


dls.show_batch((preds[1],preds[0]), max_n=8, show=True,figsize=(20,6))



In [ ]:
os.mkdir('/kaggle/working/images')


In [ ]:
pred_path = '/kaggle/working/images'


In [ ]:
test_path = '../input/ctrgb2'

In [ ]:
def get_preds_cycleGAN2(learn,test_path,pred_path,bs=4,num_workers=4,suffix='png'):
    """
    A prediction function that takes the Learner object `learn` with the trained model, 
    the `test_path` folder with the images to perform batch inference on, 
    and the output folder `pred_path` where the predictions will be saved, 
    with a batch size `bs`, `num_workers`,
    and suffix of the prediction images `suffix` (default='png').
    """

    assert os.path.exists(test_path)

    if not os.path.exists(pred_path):
        os.mkdir(pred_path)

    test_dl = load_dataset2(test_path,bs,num_workers)
    model = learn.model.G_B.cuda()
    for i, xb in tqdm.tqdm(enumerate(test_dl),total=len(test_dl)):
        fn, im = xb
        preds = (model(im.cuda())/2 + 0.5)
        for i in range(len(fn)):
            new_fn = os.path.join(pred_path,'.'.join([os.path.basename(fn[i]).split('.')[0]+'',suffix]))
            torchvision.utils.save_image(preds[i],new_fn)

In [ ]:


get_preds_cycleGAN2(learn,test_path,pred_path,suffix='png')



In [ ]:
def load_dataset2(test_path,bs=4,num_workers=4):
    "A helper function for getting a DataLoader for images in the folder `test_path`, with batch size `bs`, and number of workers `num_workers`"
    dataset = FolderDataset(
            path=test_path,
      transforms=[torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
            
        )
    loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=bs,
            num_workers=num_workers,
            shuffle=True
        )
    return loader

In [ ]:
ls img*

In [ ]:
gc.collect()

In [ ]:


shutil.make_archive('images','zip','/kaggle/working/images')



In [ ]:


ts = get_image_files(pred_path);



In [ ]:


fig = plt.figure(1,figsize=(20,10))
columns = 4
rows = 3
for i in range(rows*columns):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(immg.open(ts[i]))
    plt.axis('off')
plt.subplots_adjust(wspace=0, hspace=0.1)
plt.show()



In [ ]:
for j in range(len(ts)):
    os.remove(str(ts[j]))
os.remove('/kaggle/working/models/ct2mri.pth')
os.rmdir('/kaggle/working/images')
os.rmdir('/kaggle/working/models')

In [50]:
patho = Path('/kaggle/working')
for p in patho.ls():
    print(p)


/kaggle/working/images
/kaggle/working/img4.png
/kaggle/working/img11.png
/kaggle/working/img13.png
/kaggle/working/img12.png
/kaggle/working/images.zip
/kaggle/working/img14.png
/kaggle/working/img9.png
/kaggle/working/__notebook_source__.ipynb
/kaggle/working/img3.png
/kaggle/working/img1.png
/kaggle/working/img2.png
/kaggle/working/img5.png
/kaggle/working/img10.png
/kaggle/working/img8.png
/kaggle/working/export.pkl
/kaggle/working/img7.png
/kaggle/working/img15.png
/kaggle/working/img6.png


In [ ]:
!unzip images.zip

In [48]:
ls

__notebook_source__.ipynb  img1.png   img13.png  img3.png  img7.png
export.pkl                 img10.png  img14.png  img4.png  img8.png
images/                    img11.png  img15.png  img5.png  img9.png
images.zip                 img12.png  img2.png   img6.png
